In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  96528      0 --:--:-- --:--:-- --:--:-- 96528
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
     |████████████████████████████████| 61kB 3.5MB/s 
Uninstalling torch-1.7.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
  Successfully uninstalled torch-1.7.0+cu101
Uninstalling torchvision-0.8.1+cu101:
  Successfully uninstalled torchvision-0.8.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp37-cp37m-linux_x86_64.whl...
Done updating TPU runtime
\ [1 files][122.6 MiB/122.6 MiB]                                                
Operation completed over 1 objects/122.6 MiB.                                    
Copying gs://

In [ ]:
!unzip drive/MyDrive/highly-probable/imgs-300w.zip 

Streaming output truncated to the last 5000 lines.
  inflating: imgs-300w/15464274_2.jpg  
  inflating: imgs-300w/16233769_5.jpg  
  inflating: imgs-300w/15093270_1.jpg  
  inflating: imgs-300w/12404917_2.jpg  
  inflating: imgs-300w/15959393_3.jpg  
  inflating: imgs-300w/15733294_2.jpg  
  inflating: imgs-300w/16181030_1.jpg  
  inflating: imgs-300w/16340654_6.jpg  
  inflating: imgs-300w/14708964_2.jpg  
  inflating: imgs-300w/16011282_2.jpg  
  inflating: imgs-300w/14968364_4.jpg  
  inflating: imgs-300w/16409913_2.jpg  
  inflating: imgs-300w/16367046_2.jpg  
  inflating: imgs-300w/16274128_4.jpg  
  inflating: imgs-300w/15705283_5.jpg  
  inflating: imgs-300w/16337843_1.jpg  
  inflating: imgs-300w/14397406_4.jpg  
  inflating: imgs-300w/13414118_2.jpg  
  inflating: imgs-300w/15796297_4.jpg  
  inflating: imgs-300w/16245478_3.jpg  
  inflating: imgs-300w/16114649_8.jpg  
  inflating: imgs-300w/16027830_5.jpg  
  inflating: imgs-300w/15841774_3.jpg  
  inflating: imgs-300w/163839

In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp37-none-any.whl size=16031 sha256=3932092acd4a386accfc24aa3fea936b645be8b414c1b82d043d06cb28766635
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [ ]:
import gc
import os
import torch
import albumentations

import numpy as np
import pandas as pd

import torch.nn as nn
from sklearn import metrics
from sklearn import model_selection
from torch.nn import functional as F

# from wtfml.engine import Engine
# from wtfml.utils import EarlyStopping
# from wtfml.data_loaders.image import ClassificationDataLoader


import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

import efficientnet_pytorch

In [ ]:
NUM_EPOCHS = 15

In [ ]:
class EfficientNet(nn.Module):
    def __init__(self):
        super(EfficientNet, self).__init__()
        self.base_model = efficientnet_pytorch.EfficientNet.from_pretrained(
            'efficientnet-b0'
        )
        self.base_model._fc = nn.Linear(
            in_features=1280, 
            out_features=23, 
            bias=True
        )
        
    def forward(self, image, targets):
        out = self.base_model(image)
        #loss = nn.BCEWithLogitsLoss()(out, targets.view(-1, 1).type_as(out))
        loss = nn.CrossEntropyLoss()(out, targets.type_as(out))
        return out, loss

In [ ]:
DATA_DIR = 'imgs-300w'

def load_df():
    df = pd.read_csv("drive/MyDrive/highly-probable/womens-clothing-imageid-5fold.csv")
    # label encoding
    df['target'] = df['mid_category'].astype('category').cat.codes

    # remove rows with missing images (9 images)
    df['image_exists'] = df['image_name'].apply(lambda x: os.path.exists(f'{DATA_DIR}/{x}'))
    df = df[df['image_exists']]

    df = df[['image_name', 'target', 'fold']]

    df = df.rename(columns={'fold':'kfold'})

    
    print(df.shape)

    return df

df = load_df()
df.head()

(277620, 3)


,image_name,target,kfold
0,16222141_1.jpg,6,0.0
1,16222141_2.jpg,6,0.0
2,16222141_3.jpg,6,0.0
3,16222141_4.jpg,6,0.0
4,16222141_5.jpg,6,0.0


In [ ]:
MX = EfficientNet()

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [ ]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [ ]:
try:
    import torch_xla.core.xla_model as xm

    _xla_available = True
except ImportError:
    _xla_available = False


class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.0001, tpu=False):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.tpu = tpu
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

        if self.tpu and not _xla_available:
            raise Exception(
                "You want to use TPUs but you dont have pytorch_xla installed"
            )

    def __call__(self, epoch_score, model, model_path):
        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.tpu:
                xm.master_print(
                    "EarlyStopping counter: {} out of {}".format(
                        self.counter, self.patience
                    )
                )
            else:
                print(
                    "EarlyStopping counter: {} out of {}".format(
                        self.counter, self.patience
                    )
                )
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            if self.tpu:
                xm.master_print(
                    "Validation score improved ({} --> {}). Saving model!".format(
                        self.val_score, epoch_score
                    )
                )
            else:
                print(
                    "Validation score improved ({} --> {}). Saving model!".format(
                        self.val_score, epoch_score
                    )
                )
            if self.tpu:
                xm.save(model.state_dict(), model_path)
            else:
                torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [ ]:
import cv2
import torch

import numpy as np

from PIL import Image
from PIL import ImageFile

try:
    import torch_xla.core.xla_model as xm

    _xla_available = True
except ImportError:
    _xla_available = False

ImageFile.LOAD_TRUNCATED_IMAGES = True


class ClassificationDataset:
    def __init__(
        self,
        image_paths,
        targets,
        resize,
        augmentations=None,
        backend="pil",
        channel_first=True,
    ):
        """
        :param image_paths: list of paths to images
        :param targets: numpy array
        :param resize: tuple or None
        :param augmentations: albumentations augmentations
        """
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
        self.backend = backend
        self.channel_first = channel_first

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        targets = self.targets[item]
        if self.backend == "pil":
            image = Image.open(self.image_paths[item])
            if self.resize is not None:
                image = image.resize(
                    (self.resize[1], self.resize[0]), resample=Image.BILINEAR
                )
            image = np.array(image)
            if self.augmentations is not None:
                augmented = self.augmentations(image=image)
                image = augmented["image"]
        elif self.backend == "cv2":
            image = cv2.imread(self.image_paths[item])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.resize is not None:
                image = cv2.resize(
                    image,
                    (self.resize[1], self.resize[0]),
                    interpolation=cv2.INTER_CUBIC,
                )
            if self.augmentations is not None:
                augmented = self.augmentations(image=image)
            image = augmented["image"]
        else:
            raise Exception("Backend not implemented")
        if self.channel_first:
            image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image),
            "targets": torch.tensor(targets),
        }


class ClassificationDataLoader:
    def __init__(
        self,
        image_paths,
        targets,
        resize,
        augmentations=None,
        backend="pil",
        channel_first=True,
    ):
        """
        :param image_paths: list of paths to images
        :param targets: numpy array
        :param resize: tuple or None
        :param augmentations: albumentations augmentations
        """
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
        self.backend = backend
        self.channel_first = channel_first
        self.dataset = ClassificationDataset(
            image_paths=self.image_paths,
            targets=self.targets,
            resize=self.resize,
            augmentations=self.augmentations,
            backend=self.backend,
            channel_first=self.channel_first,
        )

    def fetch(
        self,
        batch_size,
        num_workers,
        drop_last=False,
        shuffle=True,
        tpu=False,
        sampler=None,
    ):
        """
        :param batch_size: batch size
        :param num_workers: number of processes to use
        :param drop_last: drop the last batch?
        :param shuffle: True/False
        :param tpu: True/False, to use tpu or not
        """
        if tpu:
            sampler = torch.utils.data.distributed.DistributedSampler(
                self.dataset,
                num_replicas=xm.xrt_world_size(),
                rank=xm.get_ordinal(),
                shuffle=shuffle,
            )

        data_loader = torch.utils.data.DataLoader(
            self.dataset,
            batch_size=batch_size,
            sampler=sampler,
            drop_last=drop_last,
            num_workers=num_workers,
        )
        return data_loader

In [ ]:
import datetime
import torch
from tqdm import tqdm
 
try:
    from torch.cuda import amp
    _amp_available = True
except ImportError:
    _amp_available = False
 
try:
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.parallel_loader as pl
 
    _xla_available = True
except ImportError:
    _xla_available = False
 
 
def reduce_fn(vals):
    return sum(vals) / len(vals)
 
 
class Engine:
    def __init__(
        self,
        model,
        optimizer,
        device,
        scheduler=None,
        accumulation_steps=1,
        use_tpu=False,
        tpu_print=10,
        fp16=False,
        model_fn=None,
        use_mean_loss=False,
    ):
        """
        model_fn should take batch of data, device and model and return loss
        for example:
            def model_fn(data, device, model):
                images, targets = data
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                _, loss = model(images, targets)
                return loss
        """
        self.model = model
        self.optimizer = optimizer
        self.device = device
        self.scheduler = scheduler
        self.accumulation_steps = accumulation_steps
        self.use_tpu = use_tpu
        self.tpu_print = tpu_print
        self.model_fn = model_fn
        self.fp16 = fp16
        if self.fp16 and not _amp_available:
            raise Exception(
                "You want to use fp16 but dont have amp installed"
            )
        self.use_mean_loss = use_mean_loss
        self.scaler = None
 
        if self.use_tpu and not _xla_available:
            raise Exception(
                "You want to use TPUs but you dont have pytorch_xla installed"
            )
        if self.fp16 and use_tpu:
            raise Exception("Apex fp16 is not available when using TPUs")
        if self.fp16:
            self.scaler = amp.GradScaler()
 
    def train(self, data_loader):
        losses = AverageMeter()
        self.model.train()
        print_idx = int(len(data_loader) * self.tpu_print / 100)
        if self.accumulation_steps > 1:
            self.optimizer.zero_grad()
        if self.use_tpu:
            para_loader = pl.ParallelLoader(data_loader, [self.device])
            tk0 = para_loader.per_device_loader(self.device)
        else:
            tk0 = tqdm(data_loader, total=len(data_loader))
 
        for b_idx, data in enumerate(tk0):
            if self.accumulation_steps == 1 and b_idx == 0:
                self.optimizer.zero_grad()
 
            if self.model_fn is None:
                for key, value in data.items():
                    data[key] = value.to(self.device)
                _, loss = self.model(**data)
            else:
                if self.fp16:
                    with amp.autocast():
                        loss = self.model_fn(data, self.device, self.model)
                else:
                    loss = self.model_fn(data, self.device, self.model)
 
            if not self.use_tpu:
                with torch.set_grad_enabled(True):
                    if self.use_mean_loss:
                        loss = loss.mean()
 
                    if self.fp16:
                        self.scaler.scale(loss).backward()
                    else:
                        loss.backward()
 
                    if (b_idx + 1) % self.accumulation_steps == 0:
                        if self.fp16:
                            self.scaler.step(self.optimizer)
                        else:
                            self.optimizer.step()
 
                        if self.scheduler is not None:
                            self.scheduler.step()
 
                        if self.fp16:
                            self.scaler.update()
 
                        if b_idx > 0:
                            self.optimizer.zero_grad()
 
            else:
                loss.backward()
                xm.optimizer_step(self.optimizer)
                if self.scheduler is not None:
                    self.scheduler.step()
                if b_idx > 0:
                    self.optimizer.zero_grad()
            if self.use_tpu:
                reduced_loss = xm.mesh_reduce("loss_reduce", loss, reduce_fn)
                losses.update(reduced_loss.item(), data_loader.batch_size)
            else:
                losses.update(loss.item(), data_loader.batch_size)
 
            if not self.use_tpu:
                tk0.set_postfix(loss=losses.avg)
            else:
                # if b_idx % print_idx == 0 or b_idx == len(data_loader):
                if b_idx % 10 == 0 or b_idx == len(data_loader):
                    xm.master_print(
                        f"{datetime.datetime.now()}: Batch {b_idx} / {len(data_loader)}, loss={losses.avg}"
                    )
        if not self.use_tpu:
            tk0.close()
        return losses.avg
 
    def evaluate(self, data_loader, return_predictions=False):
        losses = AverageMeter()
        print_idx = int(len(data_loader) * self.tpu_print / 100)
        self.model.eval()
        final_predictions = []
        with torch.no_grad():
            if self.use_tpu:
                para_loader = pl.ParallelLoader(data_loader, [self.device])
                tk0 = para_loader.per_device_loader(self.device)
            else:
                tk0 = tqdm(data_loader, total=len(data_loader))
            for b_idx, data in enumerate(tk0):
                for key, value in data.items():
                    data[key] = value.to(self.device)
                if self.fp16:
                    with amp.autocast():
                        batch_preds, loss = self.model(**data)
                else:
                    batch_preds, loss = self.model(**data)
                if return_predictions:
                    final_predictions.append(batch_preds)
                if self.use_tpu:
                    reduced_loss = xm.mesh_reduce("loss_reduce", loss, reduce_fn)
                    losses.update(reduced_loss.item(), data_loader.batch_size)
                else:
                    if self.use_mean_loss:
                        loss = loss.mean()
                    losses.update(loss.item(), data_loader.batch_size)
                if not self.use_tpu:
                    tk0.set_postfix(loss=losses.avg)
                else:
                    #if b_idx % print_idx == 0 or b_idx == len(data_loader):
                    if b_idx % 10 == 0 or b_idx == len(data_loader):
                        xm.master_print(
                            f"{datetime.datetime.now()}: Batch {b_idx} / {len(data_loader)}, loss={losses.avg}"
                        )
            if not self.use_tpu:
                tk0.close()
        return losses.avg, final_predictions
 
    def predict(self, data_loader):
        self.model.eval()
        final_predictions = []
        if self.use_tpu:
            raise Exception("TPU not available for predict yet!")
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader))
            for data in tk0:
                for key, value in data.items():
                    data[key] = value.to(self.device)
                predictions, _ = self.model(**data)
                predictions = predictions.cpu()
                final_predictions.append(predictions)
        return final_predictions

In [ ]:
def train():
    training_data_path = "imgs-300w/"
    df = load_df()
    device = xm.xla_device()
    epochs = NUM_EPOCHS
    train_bs = 32
    valid_bs = 16
    fold = 0
 
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
 
    model = MX.to(device)
 
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_aug = albumentations.Compose(
        [   albumentations.CenterCrop(
                height=300,
                width=300,
            ),
         
            albumentations.Normalize(
                mean, 
                std, 
                max_pixel_value=255.0, 
                always_apply=True
            ),
            albumentations.ShiftScaleRotate(
                shift_limit=0.0625, 
                scale_limit=0.1, 
                rotate_limit=15
            ),
            albumentations.Flip(p=0.5)
        ]
    )
 
    valid_aug = albumentations.Compose(
        [
            albumentations.CenterCrop(
                height=300,
                width=300,
            ),
            albumentations.Normalize(
                mean, 
                std, 
                max_pixel_value=255.0,
                always_apply=True
            )
        ]
    )

    # df_train = df_train.iloc[:256]
    # df_valid = df_valid.iloc[:128]
 
    train_images = df_train.image_name.values.tolist()
    train_images = [
        os.path.join(training_data_path, i) for i in train_images
    ]
    train_targets = df_train.target.values
 
    valid_images = df_valid.image_name.values.tolist()
    valid_images = [
        os.path.join(training_data_path, i) for i in valid_images
    ]
    valid_targets = df_valid.target.values
 
    train_loader = ClassificationDataLoader(
        image_paths=train_images,
        targets=train_targets,
        resize=None,
        augmentations=train_aug,
    ).fetch(
        batch_size=train_bs, 
        drop_last=True, 
        num_workers=4, 
        shuffle=True, 
        tpu=True
    )
 
    valid_loader = ClassificationDataLoader(
        image_paths=valid_images,
        targets=valid_targets,
        resize=None,
        augmentations=valid_aug,
    ).fetch(
        batch_size=valid_bs, 
        drop_last=True,
        num_workers=2, 
        shuffle=False, 
        tpu=True
    )
 
        
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #     optimizer,
    #     patience=3,
    #     threshold=0.001,
    #     mode="min"
    # )

    scheduler = {'scheduler': torch.optim.lr_scheduler.OneCycleLR(
                                        optimizer,
                                        max_lr=1e-3,
                                        steps_per_epoch=int(len(train_loader)),
                                        epochs=NUM_EPOCHS,
                                        anneal_strategy="linear",
                                        final_div_factor = 30,
                                    ),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
 
    es = EarlyStopping(patience=15, mode="min", tpu=True)
    eng = Engine(model, optimizer, device=device, use_tpu=True, tpu_print=25)
 
 
 
    for epoch in range(epochs):
        train_loss = eng.train(train_loader)
        valid_loss = eng.evaluate(valid_loader)[0]
        xm.master_print(f"Epoch = {epoch}, LOSS = {valid_loss}")

        #xm.master_print(valid_loss)
 
        
        es(valid_loss, model, model_path=f"drive/MyDrive/highly-probable/results/tp1/model_fold_{fold}.bin")
        if es.early_stop:
            xm.master_print("Early stopping")
            break
        gc.collect()

In [ ]:
def _mp_fn(rank, flags):
    torch.set_default_tensor_type('torch.FloatTensor')
    a = train()

In [ ]:
FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

(277620, 3)
(277620, 3)
(277620, 3)
(277620, 3)
(277620, 3)
(277620, 3)
(277620, 3)
(277620, 3)
2021-02-25 18:24:18.308545: Batch 0 / 867, loss=3.1488289833068848
2021-02-25 18:25:01.202166: Batch 10 / 867, loss=3.1056585745377974
2021-02-25 18:25:27.562712: Batch 20 / 867, loss=3.0538849717094783
2021-02-25 18:25:48.733378: Batch 30 / 867, loss=3.0042786905842442
2021-02-25 18:26:16.473327: Batch 40 / 867, loss=2.949739595738853
2021-02-25 18:26:35.710043: Batch 50 / 867, loss=2.8967777841231404
2021-02-25 18:27:02.340262: Batch 60 / 867, loss=2.8403626621746625
2021-02-25 18:27:22.909143: Batch 70 / 867, loss=2.783126988881071
2021-02-25 18:27:49.187330: Batch 80 / 867, loss=2.724502713591964
2021-02-25 18:28:08.304576: Batch 90 / 867, loss=2.671660465198559
2021-02-25 18:28:36.552325: Batch 100 / 867, loss=2.618363345023429
2021-02-25 18:28:54.247236: Batch 110 / 867, loss=2.564646557644681
2021-02-25 18:29:20.006731: Batch 120 / 867, loss=2.515570677016392
2021-02-25 18:29:42.93708

In [ ]:
df[df.target=='']